# Исследование надёжности заёмщиков

Входные данные — статистика о платёжеспособности клиентов банка.

Задача - определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

## Оглавление

* [Шаг 1. Откройте файл с данными и изучите общую информацию.](#Шаг-1.-Откройте-файл-с-данными-и-изучите-общую-информацию.)
* [Шаг 2. Предобработка данных](#Шаг-2.-Предобработка-данных)
    * [Обработка пропусков](#Обработка-пропусков)
    * [Замена типа данных](#Замена-типа-данных)
    * [Обработка дубликатов](#Обработка-дубликатов)
    * [Лемматизация](#Лемматизация)
    * [Категоризация данных](#Категоризация-данных)
* [Шаг 3. Ответьте на вопросы](#Шаг-3.-Ответьте-на-вопросы)
    * [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#Есть-ли-зависимость-между-наличием-детей-и-возвратом-кредита-в-срок?)
    * [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?)
    * [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?)
    * [Как разные цели кредита влияют на его возврат в срок?](#Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?)
* [Шаг 4. Общий вывод](#Шаг-4.-Общий-вывод)

## Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

data.info() #вывод общей информации о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


#### Вывод

Файл с данным содержит 12 столбцов (5 целочисленных столбцов, 5 столбцов с плавающей точкой и 5 объектов) и 21525 строк данных.

На первый взгляд, названия столбцов выглядят адекватно и не содержат изъянов.

В столбцах days_employed и total_income присутствуют пропущенные значения. Причем, их количество одинаково, возможно, это важно, на это нужно будет обратить внимание при предобработке данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Просмотр общей информации

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


#### Анализ пропусков в столбцах days_employed и total_income

In [4]:
# Проверка гипотезы о том, что пропущенный значения в столбцах days_employed и total_income связаны
len(data[(data['days_employed'].isnull()) & (data['total_income'].isnull())])

2174

In [5]:
# Анализ информации о людях с пропущенными значениями
data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [6]:
data['dob_years'].unique() # Проверка уникальных значений 

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [7]:
data[data['dob_years'] == 0] # Просмотр людей с возрастом, равным 0 лет

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


In [8]:
data = data[data['dob_years'] != 0].reset_index(drop=True) # Удаление записей с значением возраста, равным 0

data['dob_years'].unique() # Проверка результатов удаления

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

In [9]:
# Функция, категоризирующая данные по группам возрастов
# dob_years - возраст, numeric
# Возвращает - категорию, string
def get_dob_years_category(dob_years):
    if dob_years < 20:
        return '< 20'
    if dob_years > 20 and dob_years <= 24:
        return '20 - 24'
    if dob_years > 24 and dob_years <= 29:
        return '25 - 29'
    if dob_years > 29 and dob_years <= 34:
        return '30 - 34'
    if dob_years > 34 and dob_years <= 39:
        return '35 - 39'
    if dob_years > 39 and dob_years <= 44:
        return '40 - 44'
    if dob_years > 44 and dob_years <= 49:
        return '45 - 49'
    if dob_years > 49 and dob_years <= 54:
        return '50 - 54'
    if dob_years > 54 and dob_years <= 59:
        return '55 - 59'
    if dob_years > 59 and dob_years <= 64:
        return '60 - 64'
    if dob_years > 64 and dob_years <= 69:
        return '65 - 69'
    return '>= 70'

# Создание нового столбца dob_years_category с категориями возрастов
data['dob_years_category'] = data['dob_years'].apply(get_dob_years_category)

# Проверка результатов категоризации возрастов
data.groupby('dob_years_category')['dob_years'].value_counts()

dob_years_category  dob_years
20 - 24             24           264
                    23           254
                    22           183
                    21           111
25 - 29             29           545
                    28           503
                    27           493
                    26           408
                    25           357
30 - 34             34           603
                    33           581
                    31           560
                    30           540
                    32           510
35 - 39             35           617
                    38           598
                    39           573
                    36           555
                    37           537
40 - 44             40           609
                    41           607
                    42           597
                    44           547
                    43           513
45 - 49             48           538
                    49           508
        

In [10]:
data['education'].unique() # Проверка уникальных значений 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [11]:
data['education_id'].unique() # Проверка уникальных значений 

array([0, 1, 2, 3, 4])

In [12]:
# Проверка соотношения идентификаторов уровня образования с их названиями
data.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  4686
              ВЫСШЕЕ                   273
              Высшее                   266
1             среднее                13691
              СРЕДНЕЕ                  770
              Среднее                  708
2             неоконченное высшее      666
              Неоконченное высшее       47
              НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
3             начальное                250
              НАЧАЛЬНОЕ                 17
              Начальное                 15
4             ученая степень             4
              УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
Name: education, dtype: int64

In [13]:
data['income_type'].unique() # Проверка уникальных значений

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [14]:
# Функция, вычисляющая медиану заработной платы человека на основе категории его возраста, уровня образования и типа занятости
# row - строка из набора данных, pd.Series
# Возвращает - медиана, float
def get_total_income_median(row):
    return data[
            (data['dob_years_category'] == row['dob_years_category']) &
            (data['education_id'] == row['education_id']) &
            (data['income_type'] == row['income_type']) &
            (data['total_income'].isnull() == False)
        ]['total_income'].median()

# Присвоение пропущенным значениям total_income медианных значений по категориям
data.loc[data['total_income'].isnull(), 'total_income'] = data[data['total_income'].isnull()].apply(get_total_income_median, axis=1)

# Проверка результатов присвоения
data[data['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
1296,1,NaN,70,начальное,3,гражданский брак,1,F,сотрудник,0,NaN,операции с коммерческой недвижимостью,>= 70
5912,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,55 - 59
8104,0,NaN,64,начальное,3,гражданский брак,1,F,госслужащий,0,NaN,сыграть свадьбу,60 - 64


In [15]:
# Удаление записей с пропусками в столбце total_income, для которых невозможно найти подходящуюю категорию
data.dropna(subset=['total_income'], inplace=True)
data.reset_index(drop=True)

# Проверка результатов удаления
len(data[data['total_income'].isnull()])

0

In [16]:
data[data['days_employed'] == 0]['days_employed'].count() # Подсчет количества значений, равных 0

0

In [17]:
data = data.fillna(0) # Заполнение пропусков нулями

# Проверка результатов заполнения
len(data[(data['days_employed'].isnull()) | (data['total_income'].isnull())])

0

#### Удаление аномалий в столбце children

In [18]:
data['children'].value_counts() # Проверка уникальных значений 

 0     14078
 1      4801
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [19]:
# Удаление записей с аномальными значениями
data = data[(data['children'] != 20) & (data['children'] != -1)].reset_index(drop=True) 

data['children'].value_counts() # Проверка результатов удаления

0    14078
1     4801
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

#### Исправление отрицательных значений в столбце days_employed

In [20]:
data['days_employed'].value_counts() # Проверка уникальных значений 

 0.000000         2149
-8437.673028         1
 394214.527044       1
-769.717438          1
-3963.590317         1
                  ... 
 390769.069031       1
 341277.435955       1
 348798.745616       1
-2128.237967         1
-1984.507589         1
Name: days_employed, Length: 19151, dtype: int64

In [21]:
data[data['days_employed'] < 0]['days_employed'].count() # Подсчет количества отрицательных значений

15736

In [22]:
# Исправлений записей с отрицательными значениями
data.loc[data['days_employed'] < 0, 'days_employed'] = data[data['days_employed'] < 0]['days_employed'] * -1 

data[data['days_employed'] < 0]['days_employed'].count() # Проверка результатов исправления

0

In [23]:
data.head() # Дополнительная проверка результатов исправления

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40 - 44
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,35 - 39
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,30 - 34
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,30 - 34
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,50 - 54


#### Проверка остальных столбцов на аномалии

In [24]:
data['family_status'].unique() # Проверка уникальных значений 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [25]:
data['family_status_id'].unique() # Проверка уникальных значений 

array([0, 1, 2, 3, 4])

In [26]:
data['gender'].unique() # Проверка уникальных значений 

array(['F', 'M', 'XNA'], dtype=object)

In [27]:
data[data['gender'] == 'XNA'] # Подсчет количества людей с полом XNA

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
10592,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,20 - 24


In [28]:
data['debt'].unique() # Проверка уникальных значений 

array([0, 1])

In [29]:
data['total_income'].value_counts() # Проверка уникальных значений 

140316.366403    142
139542.193221    126
136319.471185    125
137796.286809    124
121063.464888    110
                ... 
215864.375057      1
254249.302632      1
149000.398440      1
172637.167134      1
82047.418899       1
Name: total_income, Length: 19211, dtype: int64

In [30]:
data[data['total_income'] < 0]['total_income'].count() # Подсчет количества отрицательных значений

0

In [31]:
data['purpose'].value_counts() # Проверка уникальных значений 

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           763
операции с недвижимостью                  670
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     643
операции с жильем                         642
покупка жилья для семьи                   639
жилье                                     636
покупка жилья                             635
недвижимость                              628
операции со своей недвижимостью           626
строительство собственной недвижимости    626
строительство недвижимости                620
строительство жилой недвижимости          619
покупка своего жилья                      618
покупка недвижимости                      616
ремонт жилью                              607
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

#### Вывод

__Пропуски в столбцах days_employed и total_income__
Была проведена проверка гипотезы о том, что пропущенный значения в столбцах days_employed и total_income связаны. Гипотеза подтвердилась - пропуски либо есть в обоих столбцах, либо их нет.

После этого были проанализированы данные людей, которые имеют пропуски в указанных столбцах. Была выдвинута гипотеза о том, что пропущены значения являются эквивалентом 0 у людей, которые никогда не работали и поэтому также не имеют дохода. Эта гипотеза не подтвердилась, так пропущенные значения имеются у людей с типами занятости, которые предполагают наличие дохода. Например, пенсионер, госслужащий или сотрудник.

Также не было обнаружено никаких других связей между пропущенными значениями и другими столбцами. В связи с этим был сделан вывод о том, что данные являются обычной ошибкой и в будущем стоит узнать об этом у поставщика данных. Природа связи пропусков в двух столбцах, а также причина их появления не была определена.

__Пропуски в столбце total_income__
Для того, чтобы не терять данные, было решено заменить пропуски в столбце total_income на медианные значения total_income людей схожих с людьми, у которых значения пропущены, по следующим признакам (которые также были предварительно проверены на наличие пропусков и других аномалий):
- категория возраста;
- уровень образования;
- тип занятости.

В ходе проверка столбца dob_year на пропуски было выявлено, что 100 записей имеют значение 0 в данном столбце, что не может быть реальным. Данные значения было принято удалить, так как они не подлежат вычислению на основе других столбцов и незначительно влияют на выборку из-за своего малого количества относительной всей выборки.

Чтобы получить категорию возраста человека была реализована функция, которая в зависимости от возраста человека относит его к одной из групп. Группы составлялись на основе статистического сборника "Социальное положение и уровень жизни населения России" 2019 (стр. 46, https://gks.ru/folder/210/document/13212), где возраст делится на группы по 5 лет.

В ходе проверка столбца education на пропуски было выявлено, что в столбце есть значения одинаковые по смыслу, но разные по написанию, также было выявлено, что это не отражено в столбце education_id, поэтому его можно использовать для поиска медианы. Процесс удаления дубликатов в столбце education описан в разделе "Обработка дубликатов".

После замены пропусков было обнаружено, что для 3 записей вычисление медианы невозможно ввиду отсутсвия идентичных по критериям записей. Данные записи было решено удалить поскольку они незначительно влияют на выборку из-за своего малого количества относительной всей выборки.

__Пропуски в столбце days_employed__
Так как данный столбец не влияет на результаты проверки основных гипотез было приняты решения: 
- Не удалять записи, так как они составляют значимую часть набора данных;
- Заполнить записи с пропущенными значениями нулями, так как невозможно выявить какую-либо закономерность для их заполнения. Это никак не повлияет на качество данных с точки зрения проверки основных гипотез, но необходимо для дальнейшей обработки - приведения к целочисленному типу данных колонок.

__Удаление аномалий в столбце children__
В столбце children были обнаружены значения "-20" и "-1", что не может являться реальным количеством детей. В связи с этим записи, где в столбце children указаны значения "-20" и "-1" были удалены.

__Исправление отрицательных значений в столбце days_employed__
В столбце days_employed было обнаружены отрицательные значения, которые составляют более 3/4 все значений. Так как отрицательный стаж невозможен, а количество таких значений весьма велико, был сделан вывод, что это ошибка ввода данных. Значения были заменены на положительные. Стоит указать поставщику данных на данную ошибку.

__Проверка остальных столбцов на аномалии__
В остальных столбцах была найдена только 1 аномалия:
В столбце gendre у одного человека указано значение "XNA". В связи с тем, что
- в современном обществе существуют люди, которые не относят себе ни к одному из существующих гендеров;
- количество этих людей очень мало по сравнению с общей численностью человечества;
- количество людей в данных в гендером "XNA" очень мало по сравнению с количество людей в выборке

был сделан вывод, что это может быть один из таких людей и поэтому данная аномалий не требует исправления.

### Замена типа данных

In [32]:
# Анализ данных
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21299 entries, 0 to 21298
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21299 non-null  int64  
 1   days_employed       21299 non-null  float64
 2   dob_years           21299 non-null  int64  
 3   education           21299 non-null  object 
 4   education_id        21299 non-null  int64  
 5   family_status       21299 non-null  object 
 6   family_status_id    21299 non-null  int64  
 7   gender              21299 non-null  object 
 8   income_type         21299 non-null  object 
 9   debt                21299 non-null  int64  
 10  total_income        21299 non-null  float64
 11  purpose             21299 non-null  object 
 12  dob_years_category  21299 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40 - 44
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,35 - 39
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,30 - 34
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,30 - 34
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,50 - 54


In [33]:
# Приведение к целочисленному типу данных
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [34]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21299 entries, 0 to 21298
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   children            21299 non-null  int64 
 1   days_employed       21299 non-null  int64 
 2   dob_years           21299 non-null  int64 
 3   education           21299 non-null  object
 4   education_id        21299 non-null  int64 
 5   family_status       21299 non-null  object
 6   family_status_id    21299 non-null  int64 
 7   gender              21299 non-null  object
 8   income_type         21299 non-null  object
 9   debt                21299 non-null  int64 
 10  total_income        21299 non-null  int64 
 11  purpose             21299 non-null  object
 12  dob_years_category  21299 non-null  object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,40 - 44
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,35 - 39
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,30 - 34
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,30 - 34
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,50 - 54


#### Вывод

В наборе данных 2 столбца из всех имеют вещественный тип - days_employed и total_income. 

Дробные части дней стажа и зарплаты не несут в себе информационной нагрузки ввиду того, что являются очень маленькой по сравнению с целыми частями. Поэтому было принято решение привести указанные столбцы к целочисленному типу с целью экономии памяти.

Данная процедура была осуществлена с помощью функции astype() библиотеки Pandas, так как приведение происходило от одного численного типа к другому, а не от строки к численному типу.

### Обработка дубликатов

#### Дубликаты в столбце education

In [35]:
data['education'].value_counts() # Проверка уникальных значений 

среднее                13609
высшее                  4665
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                248
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [36]:
# Проверка соотношения идентификаторов уровня образования с их названиями
data.groupby('education_id')['education'].value_counts()

education_id  education          
0             высшее                  4665
              ВЫСШЕЕ                   270
              Высшее                   266
1             среднее                13609
              СРЕДНЕЕ                  764
              Среднее                  700
2             неоконченное высшее      663
              Неоконченное высшее       47
              НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
3             начальное                248
              НАЧАЛЬНОЕ                 17
              Начальное                 15
4             ученая степень             4
              УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
Name: education, dtype: int64

In [37]:
data['education'] = data['education'].str.lower() # Удаление дубликатов путем приведения к нижнему регистру

data['education'].value_counts() # Проверка результатов удаления дубликатов

среднее                15073
высшее                  5201
неоконченное высшее      739
начальное                280
ученая степень             6
Name: education, dtype: int64

#### Общие дубликаты

In [38]:
# Проверка на дубликаты в наборе данных
data.duplicated().sum()

71

In [39]:
# Удаление дубликатов
data = data.drop_duplicates().reset_index(drop=True)

In [40]:
# Проверка результатов удаления
data.duplicated().sum()

0

#### Вывод

__Дубликаты в столбце education__
В ходе поиска дополнительных аномалий в разделе поиска пропусков было обнаружено, что в столбце education есть значения одинаковые по смыслу, но разные по написанию.

Для поиска дубликатов были проведены:
- дополнительный анализ уникальных значений с помощью функции value_counts(), который более наглядно подтвердил наличие дубликатов;
- группировка названий уровней образования по идентификатору с целью проверки необходимости устранения дубликатов в идентификаторах. Проверка показала, что такой необходимости не стоит.

После этого дубликаты были устранены путем приведения названий уровней образования к нижнему регистру.

Возникновение подобных дубликатов возможно из-за того, что данные собирались из разных источников, например, баз данных, в которых при создании таблицы были указаны одинаковые идентификаторы, но по-разному вводились названия уровней образования. После чего данные из разных БД были склеены в один набор данных.
Маловероятно, что дубликаты возникли из-за того, что данные вводились вручную, т.к. тогда в названиях бы однозначно содержались другие ошибки, а в данном случае видно, что уровень образования выбирался из списка.

__Общие дубликаты__
После удаления дубликатов в столбце education была проведена проверка всего набора данных на наличие полных дубликатов в записях. 

Проверка осуществлялась с помощью функции duplicated(). Был выявлен 71 дубликат. Дубликаты были удалены с помощью функции drop_duplicates().

Появление таких дубликатов возможно по причине того, что данные могут быть собраны из разных банков, в которые обращался клиент для получения кредита. И в связи с этим был внесен в общий набор данных несоколько раз.

### Лемматизация

In [41]:
data['purpose'].value_counts() # Проверка уникальных значений 

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           754
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     642
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      613
ремонт жилью                              602
покупка жилой недвижимости                598
на покупку своего автомобиля              501
заняться высшим образованием      

In [42]:
# Импорт функции Mystem из библиотеки 
from pymystem3 import Mystem

m = Mystem()

In [43]:
# Функция для лемматизации цели получения кредита
# purpose - цель получения кредита, string
# Возвращает - лемматизированная цель получения кредита, string
def lemmatize_purpose(purpose):
    return ''.join(m.lemmatize(purpose)[:-1])

In [44]:
data['purpose'].apply(lemmatize_purpose).value_counts() # Проверка результатов лемматизации

автомобиль                                961
свадьба                                   785
на проведение свадьба                     759
сыграть свадьба                           754
операция с недвижимость                   669
покупка коммерческий недвижимость         655
покупка жилье для сдача                   647
операция с коммерческий недвижимость      642
операция с жилье                          641
покупка жилье для семья                   636
жилье                                     635
покупка жилье                             634
недвижимость                              627
строительство собственный недвижимость    626
операция со свой недвижимость             623
строительство недвижимость                619
покупка свой жилье                        618
строительство жилой недвижимость          617
покупка недвижимость                      613
ремонт жилье                              602
покупка жилой недвижимость                598
на покупка свой автомобиль        

#### Вывод

Для лемматизации столбца purpose была написана функция lemmatize_purpose(), которая:
1. Принимает на вход строку с целью получения кредита;
2. Лемматизирует ее с помощью функции lemmatize() из библиотеки pymystem3;
3. Убирает символ переноса строки из списка с леммами, так как он не несет никакой ценности;
4. Соединяет полученный список обратно в строку;
5. Возвращает составленную из лемм строку.

### Категоризация данных

In [45]:
# Функция, проверяющая наличие детей у клиента
# amount - количество детей, numeric
# Возвращает - наличие детей, bool
def get_having_children(amount):
    if amount != 0:
        return True
    
    return False

# Создание столбца, категорезирующего наличие детей у клиента
data['have_children'] = data['children'].apply(get_having_children)

# Проверка результатов категорезации
data.groupby('have_children')['children'].value_counts()

have_children  children
False          0           14020
True           1            4791
               2            2039
               3             328
               4              41
               5               9
Name: children, dtype: int64

In [46]:
# Функция, определяющая тип семьи клиента
# amount - количество детей, numeric
# Возвращает - тип семьи, string
def get_having_children_type(amount):
    if amount == 0:
        return 'детей нет'
    elif amount in [1, 2]:
        return 'малодетная семья'
    return 'многодетная семья'

# Создание столбца, категорезирующего тип семьи клиента
data['have_children_type'] = data['children'].apply(get_having_children_type)

# Проверка результатов категорезации
data.groupby('have_children_type')['children'].value_counts()

have_children_type  children
детей нет           0           14020
малодетная семья    1            4791
                    2            2039
многодетная семья   3             328
                    4              41
                    5               9
Name: children, dtype: int64

In [47]:
# Проверка соотвествия между столбцами family_status_id и family_status
data.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12212
1                 гражданский брак          4111
2                 вдовец / вдова             946
3                 в разводе                 1179
4                 Не женат / не замужем     2780
Name: family_status, dtype: int64

In [48]:
# Функция, определяющая в браке ли клиент
# status - семейное положение, numeric
# Возвращает - в браке ли клиент, bool
def get_marrige_type(status):
    if status in [0, 1]:
        return True
    return False

# Создание столбца, категорезирующего статус брака клиента
data['married'] = data['family_status_id'].apply(get_marrige_type)

# Проверка результатов категорезации
data.groupby('married')['family_status'].value_counts()

married  family_status        
False    Не женат / не замужем     2780
         в разводе                 1179
         вдовец / вдова             946
True     женат / замужем          12212
         гражданский брак          4111
Name: family_status, dtype: int64

In [49]:
# Функция, определяющая категорию уровня дохода клиента
# income - доход, numeric
# Возвращает - категория уровня дохода, string
def get_total_income_type(income):
    if income < 50000:
        return '< 50 тыс.'
    elif income < 100000:
        return '50 - 100 тыс.'
    elif income < 150000:
        return '100 - 150 тыс.'
    elif income < 300000:
        return '150 - 300 тыс.'
    elif income < 1000000:
        return '300 тыс. - 1 млн.'
    return '> 1 млн.'

# Создание столбца, категорезирующего уровень дохода клиента
data['total_income_type'] = data['total_income'].apply(get_total_income_type)

# Проверка результатов категорезации
data.groupby('total_income_type')['total_income'].count()

total_income_type
100 - 150 тыс.       6931
150 - 300 тыс.       8402
300 тыс. - 1 млн.    1444
50 - 100 тыс.        4057
< 50 тыс.             369
> 1 млн.               25
Name: total_income, dtype: int64

In [50]:
# Функция, определяющая категорию цели получения кредита
# purpose - цель получения кредита, string
# Возвращает - категория цели получения кредита, string
def get_purpose_category(purpose):
    lemmatized_purpose = m.lemmatize(purpose)
    
    if 'недвижимость' in lemmatized_purpose:
        if 'коммерческий' in lemmatized_purpose:
            return 'коммерческая недвижимость'
        elif 'жилой' in lemmatized_purpose:
            return 'жилье'
        else:
            return 'недвижимость'
    elif 'жилье' in lemmatized_purpose:
        return 'жилье'
    elif 'автомобиль' in lemmatized_purpose:
        return 'автомобиль'
    elif 'образование' in lemmatized_purpose:
        return 'образование'
    elif 'свадьба' in lemmatized_purpose:
        return 'свадьба'

# Создание столбца, категорезирующего цель получения кредита
data['purpose_category'] = data['purpose'].apply(get_purpose_category)

# Проверка результатов категорезации
data.groupby('purpose_category')['purpose'].value_counts()

purpose_category           purpose                               
автомобиль                 на покупку своего автомобиля              501
                           автомобиль                                487
                           сделка с подержанным автомобилем          479
                           автомобили                                474
                           на покупку подержанного автомобиля        471
                           свой автомобиль                           470
                           на покупку автомобиля                     465
                           приобретение автомобиля                   457
                           сделка с автомобилем                      454
жилье                      покупка жилья для сдачи                   647
                           операции с жильем                         641
                           покупка жилья для семьи                   636
                           жилье                          

In [51]:
# Проверка результатов категоризации данных
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category,have_children,have_children_type,married,total_income_type,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,40 - 44,True,малодетная семья,True,150 - 300 тыс.,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,35 - 39,True,малодетная семья,True,100 - 150 тыс.,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,30 - 34,False,детей нет,True,100 - 150 тыс.,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,30 - 34,True,многодетная семья,True,150 - 300 тыс.,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,50 - 54,False,детей нет,True,150 - 300 тыс.,свадьба


#### Вывод

Для более точной проверки основных гипотез на шаге 3 данные были категорезированы следующими способами:
- Наличие детей: если у клиента 0 детей - False, если больше 0 - True;
- Тип семьи: 0 детей - детей нет, 1 или 2 ребенка - малодетная семья, более 2 детей - многодетная семья;
- Статус "в браке ли клиент": тип брака "женат / замужем" и "гражданский брак" - True, "вдовец / вдова", "в разводе" или "Не женат / не замужем" - False;
- Уровень дохода клиента: менее 50 тыс., 50 - 100 тыс., 100 - 150 тыс., 150 - 300 тыс., 300 тыс. - 1 млн., более 1 млн.;
- Цель получения кредита: на операции с недвижимостью - 'недвижимостью', с коммерческой недвижимостью - 'коммерческая недвижимость', с жильем - 'жилье', с автомобилем - 'автомобиль', на образование - 'образование', на проведение свадьбы - 'свадьба'

Категоризация уровня дохода клиента проведена исходя из представления о классах доходов людей автора т.к. не удалось найти статистические данные для более точной категоризации.

Категоризация по цели получения кредита осуществляется на основе лемматизции каждой цели и проверки наличия в ней одной из ключевых лемм. Для недвижимости категоризация более сложная - недвижимость разделяется на безкатегорийную, коммерческую и жилую, что, в теории, может позволить более точно определить влияние на возврат кредита в срок.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [52]:
# Формирование сводной таблицы
children_pivot = data.pivot_table(index=['children'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
children_pivot['%'] = children_pivot[1] / (children_pivot[0] + children_pivot[1]) * 100

# Вывод результатов
children_pivot

debt,0,1,%
children,,,
0,12962,1058,7.546362
1,4350,441,9.204759
2,1845,194,9.514468
3,301,27,8.231707
4,37,4,9.756098
5,9,0,0.000000


In [53]:
# Формирование сводной таблицы
have_children_pivot = data.pivot_table(index=['have_children'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
have_children_pivot['%'] = have_children_pivot[1] / (have_children_pivot[0] + have_children_pivot[1]) * 100

# Вывод результатов
have_children_pivot

debt,0,1,%
have_children,,,
False,12962,1058,7.546362
True,6542,666,9.239734


In [54]:
# Формирование сводной таблицы
have_children_type_pivot = data.pivot_table(index=['have_children_type'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
have_children_type_pivot['%'] = have_children_type_pivot[1] / (have_children_type_pivot[0] + have_children_type_pivot[1]) * 100

# Вывод результатов
have_children_type_pivot

debt,0,1,%
have_children_type,,,
детей нет,12962,1058,7.546362
малодетная семья,6195,635,9.297218
многодетная семья,347,31,8.201058


#### Вывод

Были сформированы сводные таблицы по 3 категориям наличия детей:
- количество детей;
- наличие детей;
- тип семьи (без детей, малодетная и многодетная).

Был рассчитан показательно процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов для каждой категории.

При категоризации по количеству детей прослеживается небольшая зависимость, заключающая с том, что клиенты без детей чуть чаще возвращают кредиты вовремя. Это подтверждается категоризацией клиентов по наличию детей. 
Категоризации по типу семьи также подтверждает это, а также демонстрирует, что многодетные семьи чуть чаще возвращают кредит, чем малодетные.

В целом же разница в 1 - 2% между разными категориями наличия детей является не значимой и позволяет сделать вывод, что 
__гипотеза о существовании зависимости между наличием детей и возвратом кредита в срок отвергается.__

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [55]:
# Формирование сводной таблицы
family_status_pivot = data.pivot_table(index=['family_status'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
family_status_pivot['%'] = family_status_pivot[1] / (family_status_pivot[0] + family_status_pivot[1]) * 100

# Вывод результатов
family_status_pivot.sort_values('%', ascending=False)

debt,0,1,%
family_status,,,
Не женат / не замужем,2508,272,9.784173
гражданский брак,3728,383,9.316468
женат / замужем,11289,923,7.558140
в разводе,1095,84,7.124682
вдовец / вдова,884,62,6.553911


In [56]:
# Формирование сводной таблицы
married_pivot = data.pivot_table(index=['married'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
married_pivot['%'] = married_pivot[1] / (married_pivot[0] + married_pivot[1]) * 100

# Вывод результатов
married_pivot

debt,0,1,%
married,,,
False,4487,418,8.521916
True,15017,1306,8.000980


#### Вывод

Были сформированы сводные таблицы по 2 категориям семейного положения:
- семейное положение;
- в браке ли клиент.

Был рассчитан показательно процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов для каждой категории.

В данном случае не вполне понятно отличие типов "гражданский брак" и "женат / замужем". Данный вопрос стоит уточнить у поставщика данных.

При категоризации по семейному положению прослеживается тенденция, что реже всех вовремя отдают кредит клиенты, никогда не находившиеся в браке или находящиеся в гражданском браке (предположительно, сожительствующие, но не скрепившие отношения юридически). Клиенты, находящиеся в браке, а также уже разведенные чуть чаще отдают кредит вовремя. Чаще всех вовремя же отдают кредит клиенты, имеющие статус "вдовец / вдова".

Но при категоризации людей по признаку "в браке ли клиент" в данный момент разница практически отсутствует.

В целом же сколько-нибудь значимое отличие присутствует только у клиентов, имеющих статус "вдовец / вдова". Но таких людей в принципе очень мало, относительно всей выборки, и, возможно, это продиктовано их возрастом, что требует дополнительного исследования. Это позволяет сделать вывод, что 
__гипотеза о существовании зависимости между семейным положением и возвратом кредита в срок отвергается.__

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [57]:
# Формирование сводной таблицы
total_income_type_pivot = data.pivot_table(index=['total_income_type'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
total_income_type_pivot['%'] = total_income_type_pivot[1] / (total_income_type_pivot[0] + total_income_type_pivot[1]) * 100

# Вывод результатов
total_income_type_pivot.sort_values('%')

debt,0,1,%
total_income_type,,,
< 50 тыс.,346,23,6.233062
300 тыс. - 1 млн.,1340,104,7.202216
150 - 300 тыс.,7750,652,7.760057
> 1 млн.,23,2,8.000000
50 - 100 тыс.,3727,330,8.134089
100 - 150 тыс.,6318,613,8.844323


#### Вывод

Была сформирована сводная таблица по типу уровня дохода.

Был рассчитан показательно процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов для каждого типа.

Из полученной информации можно сделать следующие выводы:
1. Чаще всего вовремя отдают кредиты клиенты, имеющие доход менее 50 тыс. Возможно, это объясняется маленькими суммами кредитов, которые выдаются данной категории клиентов;
2. Реже всех вовремя отдают кредиты клиенты, имеющие средний доход - 50 - 150 тыс. Возможно, это объясняется значительными нуждами данной категории клиентов (покупка недвижимости, личного транспорта, наличие детей и т.п.), но для подтверждения этой гипотезы необходимо дополнительное исследование;
3. Клиенты, имеющие высокий уровень дохода - от 150 тыс., находятся посередине. Возможно, это объясняется схожими нуждами с категорией клиентов со средним доходом, но наличием большего уровня дохода, что позволяет им чаще вовремя производить погашение кредита.

В целом же разница в 1 - 2% между разными типами уровня дохода является не значимой и позволяет сделать вывод, что
__гипотеза о существовании зависимости между уровнем дохода и возвратом кредита в срок отвергается.__

### Как разные цели кредита влияют на его возврат в срок?

In [58]:
# Формирование сводной таблицы
purpose_category_pivot = data.pivot_table(index=['purpose_category'], columns='debt', values='total_income', aggfunc='count', fill_value=0)

# Расчет процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов по категориям
purpose_category_pivot['%'] = purpose_category_pivot[1] / (purpose_category_pivot[0] + purpose_category_pivot[1]) * 100

# Вывод результатов
purpose_category_pivot.sort_values('%')

debt,0,1,%
purpose_category,,,
жилье,5233,395,7.018479
недвижимость,3493,284,7.519195
коммерческая недвижимость,1199,98,7.555898
свадьба,2117,181,7.876414
образование,3601,369,9.294710
автомобиль,3861,397,9.323626


#### Вывод

Была сформирована сводная таблица по категории цели получения кредитов.

Был рассчитан показательно процентного соотношения количества клиентов, не вернувших долг вовремя к общему количеству клиентов для каждого типа.

Разница в 1 - 2% между разными целями получения кредитов является не значимой и позволяет сделать вывод, что
__гипотеза о существовании зависимости целью кредита и возвратом кредита в срок отвергается.__

## Шаг 4. Общий вывод

После проверки всех гипотез можно сделать вывод, что зависимости между
- наличием детей и возвратом кредита в срок;
- семейным положением и возвратом кредита в срок;
- уровнем дохода и возвратом кредита в срок;
- цель получения кредита и возвратом кредита в срок
отсутствуют.

Также необходимо заметить, что в разных категория представлено количество людей, значительно отличающееся друг от друга (вплоть до десятков и сотен раз). Поэтому, возможно, стоит провести подобное исследование при наличие более равномерно распределенных данных.

## Чек-лист готовности проекта

- [x]  открыт файл;
- [x]  файл изучен;


- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;


- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;


- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;


- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;


- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;


- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";


- [x]  в каждом этапе есть выводы;


- [x]  есть общий вывод.